## Facundo Facio's homework 1

### Question 1: [Index] S&P 500 Stocks Added to the Index

In [1]:
import pandas as pd

In [33]:
wikipedia_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
df =pd.read_html(
    io=wikipedia_url, 
    attrs={"id": "constituents"},
    parse_dates=["Date added"],
)[0]
df = df[["Symbol", "Security", "Date added"]].rename(
    columns={
        "Symbol": "ticker",
        "Security": "name",
        "Date added": "date_added",
    }
)
df.head()

,ticker,name,date_added
0,MMM,3M,1957-03-04
1,AOS,A. O. Smith,2017-07-26
2,ABT,Abbott Laboratories,1957-03-04
3,ABBV,AbbVie,2012-12-31
4,ACN,Accenture,2011-07-06


#### Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)?

In [48]:
# group by year added and count the number of companies added each year
df["year_added"] = df["date_added"].dt.year
df_yearly = df.groupby("year_added").size().reset_index(name="count")

higher_additions = df_yearly[df_yearly["year_added"] != 1957].sort_values(
    by="count", ascending=False).iloc[0]

print(
    f"The year with the highest number of S&P 500 additions is "
    f"{higher_additions['year_added']} with {higher_additions['count']} additions."
    )

The year with the highest number of S&P 500 additions is 2017 with 23 additions.


#### Additional: How many current S&P 500 stocks have been in the index for more than 20 years?

In [173]:
df = pd.read_html(
    io=wikipedia_url,
    attrs={"id": "changes"},
    parse_dates=[('Date', 'Date')],
)[0]
df = df[[("Date", "Date"), ("Added", "Ticker"), ("Removed", "Ticker")]].rename(
    columns={
        "Date": "date",
        "Added": "added",
        "Removed": "removed",
    }
)
df.columns = df.columns.droplevel(1)
df.head()

,date,added,removed
0,2025-05-19,COIN,DFS
1,2025-03-24,DASH,BWA
2,2025-03-24,TKO,TFX
3,2025-03-24,WSM,CE
4,2025-03-24,EXE,FMC


In [176]:
# reformat the dataframe to have one row per ticket and two columns: 'added' and 'removed'
df_melted = df.melt(
    id_vars=["date"],
    value_vars=["added", "removed"],
    var_name="action",
    value_name="ticker"
)
df_melted.head()

,date,action,ticker
0,2025-05-19,added,COIN
1,2025-03-24,added,DASH
2,2025-03-24,added,TKO
3,2025-03-24,added,WSM
4,2025-03-24,added,EXE


In [175]:
# Separate and sort
added = df_melted[df_melted['action'] == 'added'].sort_values(
    ['ticker', 'date']).copy()
removed = df_melted[df_melted['action'] == 'removed'].sort_values(
    ['ticker', 'date']).copy()

# Assign a group number per ticker for pairing
added['cycle'] = added.groupby('ticker').cumcount()
removed['cycle'] = removed.groupby('ticker').cumcount()

# Merge on ticker + cycle
result = pd.merge(
    added[['ticker', 'date']].assign(cycle=added['cycle']),
    removed[['ticker', 'date']].assign(cycle=removed['cycle']),
    on=['ticker', 'cycle'],
    how='left',
    suffixes=('_added', '_removed')
).drop(columns='cycle'
       ).dropna(subset=["ticker"])

# print(f"any column has nan?:\n{result.isna().any()}\n")

# for tickers without a removal date, set it to today's date
result['date_removed'] = result['date_removed'].fillna(pd.Timestamp.today())

# print(f"any column has nan?:\n{result.isna().any()}\n")

# add a column for the number of days in the S&P 500
result['days_in_sp500'] = (
    result['date_removed'] - result['date_added']
).dt.days

# group by ticker and get tue sum of days_in_sp500
result_grouped = result.groupby('ticker').agg(
    total_days_in_sp500=('days_in_sp500', 'sum')
).reset_index()
# sort by total_days_in_sp500 in descending order
result_grouped = result_grouped.sort_values(
    by='total_days_in_sp500', ascending=False
)
# get total_years_in_sp500
result_grouped['total_years_in_sp500'] = (
    result_grouped['total_days_in_sp500'] / 365.25
).round(2)
# count the number of tickers with more than 20 years in the S&P 500
count_more_than_20_years = (
    result_grouped[result_grouped['total_years_in_sp500'] > 20].shape[0]
)
print(
    f"There are {count_more_than_20_years} tickers with more than 20 years in the S&P 500."
)

There are 10 tickers with more than 20 years in the S&P 500.


In [159]:
result['date_removed'].iloc[0]

Timestamp('2024-09-23 00:00:00')

In [158]:
result['date_added'].iloc[0]

Timestamp('2015-03-23 00:00:00')

In [107]:
# for the same ticker, substract the removed from the added dates
# df_melted["date"] = pd.to_datetime(df_melted["date"])
df_melted.sort_values(by=["ticker", "date"])


,date,action,ticker
551,2016-11-01,removed,AA
228,2015-03-23,added,AAL
383,2024-09-23,removed,AAL
221,2015-07-08,added,AAP
405,2023-08-25,removed,AAP
...,...,...,...
392,2024-04-02,removed,NaN
18,2024-04-03,added,NaN
19,2024-04-03,added,NaN
382,2024-09-30,removed,NaN


In [ ]:
df_melted2 = df_melted.melt(
    id_vars=["action", "ticker"],
    value_vars=["date"],
    var_name
)

In [96]:
df_melted2 = df.melt(
    id_vars=["added", "removed"],
    value_vars=["date"],
    var_name="ticker",
    value_name="date"
)

ValueError: value_name (date) cannot match an element in the DataFrame columns.

### Question 2. [Macro] Indexes YTD (as of 1 May 2025)